# BioConsult: Web-збір + аналіз + експорт у data/raw

Цей ноутбук:
- шукає інформацію в Wikipedia, PubMed, OpenAlex, Crossref
- завантажує сторінки та витягує основний текст
- формує просту «нормальну» відповідь без показу конспектів
- експортує файли у `../data/raw/` для твого GitHub Pages проєкту

> Запусти клітинки по черзі. В кінці зміни `query` та перезапусти останній блок.

In [ ]:
!pip -q install requests beautifulsoup4 trafilatura lxml tiktoken

In [ ]:

import os, re, json, textwrap, time
from dataclasses import dataclass
from typing import List, Optional, Dict

import requests
from bs4 import BeautifulSoup
import trafilatura

HEADERS = {
    "User-Agent": "BioConsultStudyBot/1.0 (educational)"
}

# Якщо ноутбук лежить у /notebooks, то DATA_DIR = ../data/raw
DATA_DIR = os.path.join("..", "data", "raw")
os.makedirs(DATA_DIR, exist_ok=True)

def clean_text(t: str) -> str:
    t = re.sub(r"\s+", " ", t or "").strip()
    return t

def truncate(t: str, n=3500) -> str:
    return t[:n] + ("…" if len(t) > n else "")


## Пошук по джерелах

In [ ]:

def search_wikipedia(query: str, lang: str = "uk", limit: int = 5) -> List[Dict]:
    url = f"https://{lang}.wikipedia.org/w/api.php"
    params = {
        "action": "query",
        "list": "search",
        "srsearch": query,
        "format": "json",
        "utf8": 1
    }
    r = requests.get(url, params=params, headers=HEADERS, timeout=20)
    r.raise_for_status()
    data = r.json()
    out = []
    for item in data.get("query", {}).get("search", [])[:limit]:
        title = item["title"]
        page_url = f"https://{lang}.wikipedia.org/wiki/{title.replace(' ', '_')}"
        out.append({"source": "wikipedia", "title": title, "url": page_url})
    return out


In [ ]:

def search_pubmed(query: str, limit: int = 5) -> List[Dict]:
    esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    r = requests.get(esearch, params={
        "db": "pubmed",
        "term": query,
        "retmax": limit,
        "retmode": "json"
    }, headers=HEADERS, timeout=20)
    r.raise_for_status()
    ids = r.json().get("esearchresult", {}).get("idlist", [])
    out = []
    for pmid in ids:
        out.append({"source": "pubmed", "title": f"PubMed PMID:{pmid}", "url": f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"})
    return out


In [ ]:

def search_openalex(query: str, limit: int = 5) -> List[Dict]:
    url = "https://api.openalex.org/works"
    r = requests.get(url, params={"search": query, "per_page": limit}, headers=HEADERS, timeout=20)
    r.raise_for_status()
    data = r.json()
    out = []
    for w in data.get("results", [])[:limit]:
        title = w.get("title") or "Untitled"
        landing = w.get("primary_location", {}).get("landing_page_url") or w.get("id")
        out.append({"source": "openalex", "title": title, "url": landing})
    return out


In [ ]:

def search_crossref(query: str, limit: int = 5) -> List[Dict]:
    url = "https://api.crossref.org/works"
    r = requests.get(url, params={"query": query, "rows": limit}, headers=HEADERS, timeout=20)
    r.raise_for_status()
    items = r.json().get("message", {}).get("items", [])[:limit]
    out = []
    for it in items:
        title = (it.get("title") or ["Untitled"])[0]
        link = it.get("URL")
        if not link:
            continue
        out.append({"source": "crossref", "title": title, "url": link})
    return out


## Завантаження і витяг тексту

In [ ]:

def fetch_and_extract(url: str) -> str:
    if not url:
        return ""
    try:
        downloaded = trafilatura.fetch_url(url)
        if not downloaded:
            r = requests.get(url, headers=HEADERS, timeout=25)
            r.raise_for_status()
            downloaded = r.text
        text = trafilatura.extract(downloaded, include_comments=False, include_tables=False)
        return clean_text(text or "")
    except Exception:
        return ""


In [ ]:

@dataclass
class Doc:
    title: str
    url: str
    source: str
    text: str

def build_corpus(query: str, lang="uk", per_source=3) -> List[Doc]:
    hits = []
    hits += search_wikipedia(query, lang=lang, limit=per_source)
    hits += search_pubmed(query, limit=per_source)
    hits += search_openalex(query, limit=per_source)
    hits += search_crossref(query, limit=per_source)

    docs: List[Doc] = []
    for h in hits:
        text = fetch_and_extract(h["url"])
        if len(text) < 500:
            continue
        docs.append(Doc(title=h["title"], url=h["url"], source=h["source"], text=text))
        time.sleep(0.25)
    return docs


## Простий «офлайн AI»-конспект без показу фрагментів

In [ ]:

def simple_answer(query: str, docs: List[Doc], max_chars=3500) -> str:
    if not docs:
        return "Не вдалося знайти достатньо відкритих джерел під цей запит."

    blocks = []
    for d in docs[:6]:
        snippet = truncate(d.text, 900)
        blocks.append(snippet)

    merged = " ".join(blocks)
    merged = truncate(merged, max_chars)

    answer = []
    answer.append(f"Запит: {query}\n")
    answer.append("Коротко по суті:")
    answer.append(textwrap.fill(merged, width=100))
    answer.append("\nЯкщо хочеш — напиши формат: (1) визначення, (2) етапи, (3) приклади, (4) порівняння — і я перероблю.")
    return "\n\n".join(answer)


## Експорт у `../data/raw/` для GitHub Pages

In [ ]:

def export_to_txt(docs: List[Doc], out_path: str):
    lines = []
    for i, d in enumerate(docs, 1):
        lines.append(f"=== DOC #{i} ===")
        lines.append(f"TITLE: {d.title}")
        lines.append(f"SOURCE: {d.source}")
        lines.append(f"URL: {d.url}")
        lines.append("")
        lines.append(d.text)
        lines.append("")
    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

def export_answer_only(answer: str, out_path: str):
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(answer)


## Запуск

Зміни `query` і запусти. Файли збережуться у:
- `data/raw/web_notes.txt`
- `data/raw/web_answer.txt`

Потім зроби **commit+push**, щоб GitHub Pages їх підхопив.

In [ ]:

query = "мітохондрія функції етапи утворення АТФ"  # <-- змінюй тут

docs = build_corpus(query, lang="uk", per_source=3)
print("Docs:", len(docs))

out_corpus = os.path.join(DATA_DIR, "web_notes.txt")
export_to_txt(docs, out_corpus)
print("Saved:", out_corpus)

ans = simple_answer(query, docs)
out_ans = os.path.join(DATA_DIR, "web_answer.txt")
export_answer_only(ans, out_ans)
print("Saved:", out_ans)

print("\n--- ANSWER PREVIEW ---\n")
print(ans[:1200])
